In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import math

In [43]:
data_path = 'weather.csv' 
data = pd.read_csv(data_path)
if 'Unnamed: 0' in data.columns:
    data.drop('Unnamed: 0', axis=1, inplace=True)

In [44]:
#Pembersihan Data
def deteksi_outlier_iqr_manual(data):
    outlier_indices = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        Q1 = data[kolom].quantile(0.25)
        Q3 = data[kolom].quantile(0.75)
        IQR = Q3 - Q1
        batas_bawah = Q1 - 1.5 * IQR
        batas_atas = Q3 + 1.5 * IQR
        outlier_mask = (data[kolom] < batas_bawah) | (data[kolom] > batas_atas)
        outlier_indices[kolom] = data[kolom][outlier_mask].index.tolist()
    return outlier_indices

def imputasi_outlier(data, outlier_indices):
    for kolom, indices in outlier_indices.items():
        median = data[kolom].median()
        data.loc[indices, kolom] = median
    return data

In [45]:
def cari_persentil(data_terurut, persentil):
    k = (len(data_terurut) - 1) * (persentil / 100)
    f = int(k)
    c = f + 1 if k - f > 0 else f
    if f < len(data_terurut) - 1:
        return data_terurut[f] + (k - f) * (data_terurut[c] - data_terurut[f])
    else:
        return data_terurut[f]

def statistik_manual(data):
    hasil = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        nilai = data[kolom].dropna()
        data_terurut = sorted(nilai)
        n = len(data_terurut)

        mean = sum(data_terurut) / n
        median = cari_persentil(data_terurut, 50)
        variansi = sum((x - mean) ** 2 for x in data_terurut) / (n - 1)
        standar_deviasi = variansi ** 0.5

        hasil[kolom] = {
            'nilai mean': mean,
            'nilai median': median,
            'nilai standar deviasi': standar_deviasi,
            'nilai variansi': variansi,
        }
    return pd.DataFrame(hasil)

Kasus 1

Perusahaan menerima beberapa keluhan bahwa prediksi terkait nilai humidity di suatu daerah seringkali tidak tepat. Hal tersebut berakibat pada kurangnya persiapan masyarakat dalam melakukan penyesuaian kondisi termal tertentu. Dapatkah Anda mengecek apakah rata-rata nilai Humidity lebih dari 75?

In [46]:
# Deteksi dan imputasi outlier
outlier_indices_manual = deteksi_outlier_iqr_manual(data)
data = imputasi_outlier(data.copy(), outlier_indices_manual)

humidity_data = data['Humidity']

# LANGKAH 1
# Hipotesis nol (H0): Rata-rata nilai Humidity (μ) = 75
mu_0 = 75

# LANGKAH 2
# Hipotesis alternatif (H1): Rata-rata nilai Humidity (μ) > 75

# LANGKAH 3
# Tingkat signifikan (α) = 0.05 
alpha = 0.05

# LANGKAH 4
# Kalkulasi nilai T kritis untuk alpha = 0.05 (Asumsi distribusi T)
t_critical = stats.t.ppf(1 - alpha, df=len(humidity_data) - 1)

# LANGKAH 5
humidity_data = data['Humidity']

sample_mean = sum(humidity_data) / len(humidity_data)

sum_of_squares = sum([(x - sample_mean) ** 2 for x in humidity_data])

sample_std_dev = math.sqrt(sum_of_squares / (len(humidity_data) - 1))

t_statistic = (sample_mean - mu_0) / (sample_std_dev / math.sqrt(len(humidity_data)))

def t_cdf(t, df):
    x = abs(t)
    a = 0.5 * math.exp(math.lgamma((df + 1) / 2) - math.lgamma(df / 2)) * (1 + (x ** 2) / df) ** (-(df + 1) / 2)
    if t < 0:
        return a
    else:
        return 1 - a

p_value = 1 - t_cdf(t_statistic, len(humidity_data) - 1)

p_value_library = 1 - stats.t.cdf(t_statistic, df=len(humidity_data) - 1)

print("p value manual =", p_value)
print("p value library =", p_value_library)
print("Nilai kritis ketika t > ", t_critical)

#LANGKAH 6
if p_value < alpha:
    print("TOLAK H0: Rata-rata nilai Humidity lebih dari 75.")
else:
    print("Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan rata-rata nilai Humidity lebih dari 75.")


print(f"T-statistik: {t_statistic}")
print(f"P-value: {p_value_manual}")

p value manual = 1.0
p value library = 1.0
Nilai kritis ketika t >  1.645616248187262
Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan rata-rata nilai Humidity lebih dari 75.
T-statistik: -23.075432908744677
P-value: 0.9613361161137364


Kasus 2

Perusahaan mengeluarkan nilai rata-rata UV_index sebesar 3. Akan tetapi, mayoritas pelanggan mengeluhkan kulitnya terasa terbakar. Sebagai karyawan yang baik, periksalah apakah rata-rata UV_Index yang diamati memang tidak sama dengan 3 (sesuai laporan pelanggan)?

In [47]:
# LANGKAH 1
# Hipotesis nol (H0): Rata-rata UV-index = 3
mu_0 = 3

# LANGKAH 2
# Hipotesis alternatif (H1): Rata-rata UV-index != 3

# LANGKAH 3
# Tingkat signifikan (α) = 0.05 
alpha = 0.05

# LANGKAH 4
# Kalkulasi Nilai Kritis
uv_index_data = data['UV_Index']
left_t_critical = stats.t.ppf(alpha / 2, df=len(uv_index_data) - 1)
right_t_critical = stats.t.ppf(1 - alpha / 2, df=len(uv_index_data) - 1)

# LANGKAH 5
sample_mean = sum(uv_index_data) / len(uv_index_data)
sum_of_squares = sum([(x - sample_mean) ** 2 for x in uv_index_data])
sample_std_dev = math.sqrt(sum_of_squares / (len(uv_index_data) - 1))
t_statistic = (sample_mean - mu_0) / (sample_std_dev / math.sqrt(len(uv_index_data)))

def t_cdf(t, df):
    x = abs(t)
    a = 0.5 * math.exp(math.lgamma((df + 1) / 2) - math.lgamma(df / 2)) * (1 + (x ** 2) / df) ** (-(df + 1) / 2)
    if t < 0:
        return a
    else:
        return 1 - a

p_value_manual = 2 * (1 - t_cdf(abs(t_statistic), len(uv_index_data) - 1))

p_value_library = 2 * (1 - stats.t.cdf(abs(t_statistic), df=len(uv_index_data) - 1))

# LANGKAH 6
print("p value manual =", p_value_manual)
print("p value library =", p_value_library)
print("Nilai kritis ketika t < ", left_t_critical, " atau ", "Nilai kritis ketika t > ", right_t_critical)

if p_value_manual < alpha:
    print("TOLAK H0: Rata-rata nilai UV_Index tidak sama dengan 3.")
else:
    print("Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan UV_Index tidak sama dengan 3.")


print(f"T-statistik: {t_statistic}")
print(f"P-value: {p_value_manual}")

p value manual = 0.0
p value library = 0.0
Nilai kritis ketika t <  -1.9611514201705618  atau  Nilai kritis ketika t >  1.9611514201705613
TOLAK H0: Rata-rata nilai UV_Index tidak sama dengan 3.
T-statistik: 42.58523893941472
P-value: 0.0


KASUS 3

Pemerintah setempat menyarankan agar penduduk menghindari aktivitas di luar ruangan di 5 hari terakhir (asumsi: data terbaru berada pada urutan paling awal) karena nilai rata-rata pressure diprediksi berada di angka 950. Periksa apakah nilai rata-rata pressure ?

In [48]:
# Filter the pressure data for the last 5 days
pressure_data = data['Pressure'].head(5)

# LANGKAH 1
# Hipotesis nol (H0): Rata-rata nilai Pressure (μ) = 950
mu_0 = 950

# LANGKAH 2
# Hipotesis alternatif (H1): Rata-rata nilai Pressure (μ) ≠ 950

# LANGKAH 3
# Tingkat signifikan (α) = 0.05 
alpha = 0.05

# LANGKAH 4
# Kalkulasi Nilai Kritis
left_t_critical = stats.t.ppf(alpha / 2, df=len(pressure_data) - 1)
right_t_critical = stats.t.ppf(1 - alpha / 2, df=len(pressure_data) - 1)

# LANGKAH 5
sample_mean = sum(pressure_data) / len(pressure_data)
sum_of_squares = sum([(x - sample_mean) ** 2 for x in pressure_data])
sample_std_dev = math.sqrt(sum_of_squares / (len(pressure_data) - 1))
t_statistic = (sample_mean - mu_0) / (sample_std_dev / math.sqrt(len(pressure_data)))

def t_cdf(t, df):
    x = abs(t)
    a = 0.5 * math.exp(math.lgamma((df + 1) / 2) - math.lgamma(df / 2)) * (1 + (x ** 2) / df) ** (-(df + 1) / 2)
    return a

p_value_manual = 2 * (1 - t_cdf(abs(t_statistic), len(pressure_data) - 1))
p_value_library = 2 * (1 - stats.t.cdf(abs(t_statistic), df=len(pressure_data) - 1))

# LANGKAH 6
print("p value manual =", p_value_manual)
print("p value library =", p_value_library)
print("Nilai kritis ketika t < ", left_t_critical, " atau ", "Nilai kritis ketika t > ", right_t_critical)

if p_value_manual < alpha:
    print("TOLAK H0: Rata-rata nilai Pressure tidak sama dengan 950.")
else:
    print("Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan rata-rata nilai Pressure tidak sama dengan 950.")

print(f"T-statistik: {t_statistic}")
print(f"P-value: {p_value_manual}")

p value manual = 0.9613361161137364
p value library = 0.5545525495366928
Nilai kritis ketika t <  -2.7764451051977996  atau  Nilai kritis ketika t >  2.7764451051977987
Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan rata-rata nilai Pressure tidak sama dengan 950.
T-statistik: 0.6441556286115692
P-value: 0.9613361161137364
